In [1]:
import tensorflow as tf
import numpy as np
import re
import os
from sklearn.metrics import classification_report

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
with open('entities-bm-normalize-v3.txt','r') as fopen:
    texts= list(filter(None, fopen.read().split('\n')))
len(texts)

15194

In [3]:
train_texts = texts
# dataset is too small
test_texts = texts

In [4]:
word2idx = {'PAD': 0,'NUM':1,'UNK':2}
tag2idx = {'PAD': 0}
pos2idx = {'PAD': 0}
char2idx = {'PAD': 0}
word_idx = 3
tag_idx = 1
pos_idx = 1
char_idx = 1

In [5]:
def process_word(word, lower=True):
    if lower:
        word = word.lower()
    else:
        if word.isupper():
            word = word.title()
    word = re.sub('[^A-Za-z0-9\- ]+', '', word)
    if word.isdigit():
        word = 'NUM'
    return word

def read_file(f):
    global word_idx, tag_idx, char_idx, pos_idx
    words, tags, poss, X, Y, Y_pos = [], [], [], [], [], []
    for line in f:
        line = line.strip()
        if (len(line) == 0 or line.startswith("-DOCSTART-")):
            continue
        else:
            ls = line.split(' ')
            try:
                word, tag, pos = ls[0],ls[1],ls[2]
            except:
                continue
            for c in word:
                if c not in char2idx:
                    char2idx[c] = char_idx
                    char_idx += 1
            word = process_word(word)
            if len(word) < 1:
                continue
            words += [word]
            tags += [tag]
            poss += [pos]
            if word not in word2idx:
                word2idx[word] = word_idx
                word_idx += 1
            X.append(word2idx[word])
            if tag not in tag2idx:
                tag2idx[tag] = tag_idx
                tag_idx += 1
            if pos not in pos2idx:
                pos2idx[pos] = pos_idx
                pos_idx += 1
            Y.append(tag2idx[tag])
            Y_pos.append(pos2idx[pos])
                        
    return words, tags, poss, X, Y, Y_pos

In [6]:
train_words, train_tags, train_poss, train_X, train_Y, train_Y_pos = read_file(train_texts)
test_words, test_tags, test_poss, test_X, test_Y, test_Y_pos = read_file(test_texts)

In [7]:
idx2tag={idx: tag for tag, idx in tag2idx.items()}
idx2pos={idx: tag for tag, idx in pos2idx.items()}
idx2word={idx: tag for tag, idx in word2idx.items()}
batch_size = 16
dim_word = 128
dim_char = 32
dropout = 0.8
learning_rate = 1e-2
hidden_size_char = 64
hidden_size_word = 128
num_layers = 2
seq_len = 20
display_step = 200
epoch = 20

In [37]:
import json
with open('concat-settings.json','w') as fopen:
    fopen.write(json.dumps({'idx2tag':idx2tag,'idx2pos':idx2pos,'idx2word':idx2word,
           'word2idx':word2idx,'tag2idx':tag2idx,'pos2idx':pos2idx,'char2idx':char2idx}))

In [9]:
class Model:
    def __init__(self, dim_word, dim_char, dropout, learning_rate,
                 hidden_size_char, hidden_size_word, num_layers):
        
        def cells(size, reuse=False):
            return tf.nn.rnn_cell.LSTMCell(size,initializer=tf.orthogonal_initializer(),reuse=reuse)
        
        def clip_grads(loss):
            variables = tf.trainable_variables()
            grads = tf.gradients(loss, variables)
            clipped_grads, _ = tf.clip_by_global_norm(grads, 5.0)
            return zip(clipped_grads, variables)
        
        self.word_ids = tf.placeholder(tf.int32, shape=[None, None])
        self.char_ids = tf.placeholder(tf.int32, shape=[None, None, None])
        self.labels = tf.placeholder(tf.int32, shape=[None, None])
        self.labels_pos = tf.placeholder(tf.int32, shape=[None, None])
        
        self.word_embeddings = tf.Variable(tf.truncated_normal([len(word2idx), dim_word],
                                                      stddev=1.0 / np.sqrt(dim_word)))
        self.char_embeddings = tf.Variable(tf.truncated_normal([len(char2idx), dim_char],
                                                      stddev=1.0 / np.sqrt(dim_char)))
        word_embedded = tf.nn.embedding_lookup(self.word_embeddings, self.word_ids)
        char_embedded = tf.nn.embedding_lookup(self.char_embeddings, self.char_ids)
        s = tf.shape(char_embedded)
        char_embedded = tf.reshape(char_embedded, shape=[s[0]*s[1], s[-2], dim_char])
        for n in range(num_layers):
            (out_fw, out_bw), (state_fw, state_bw) = tf.nn.bidirectional_dynamic_rnn(
                cell_fw = cells(hidden_size_char),
                cell_bw = cells(hidden_size_char),
                inputs = char_embedded,
                dtype = tf.float32,
                scope = 'bidirectional_rnn_char_%d'%(n))
            char_embedded = tf.concat((out_fw, out_bw), 2)
        output = tf.reshape(char_embedded[:,-1], shape=[s[0], s[1], 2*hidden_size_char])
        word_embedded = tf.concat([word_embedded, output], axis=-1)
        word_embedded = tf.nn.dropout(word_embedded, dropout)
        
        for n in range(num_layers):
            (out_fw, out_bw), (state_fw, state_bw) = tf.nn.bidirectional_dynamic_rnn(
                cell_fw = cells(hidden_size_word),
                cell_bw = cells(hidden_size_word),
                inputs = word_embedded,
                dtype=tf.float32,
                scope = 'bidirectional_rnn_word_%d'%(n))
            word_embedded = tf.concat((out_fw, out_bw), 2)
        word_embedded = tf.nn.dropout(word_embedded, dropout)
        
        W_tag = tf.get_variable('w',shape=(2*hidden_size_word, len(idx2tag)),
                            initializer=tf.orthogonal_initializer())
        b_tag = tf.get_variable('b',shape=(len(idx2tag)),initializer=tf.zeros_initializer())
        
        W_pos = tf.get_variable('w_pos',shape=(2*hidden_size_word, len(idx2pos)),
                            initializer=tf.orthogonal_initializer())
        b_pos = tf.get_variable('b_pos',shape=(len(idx2pos)),initializer=tf.zeros_initializer())
        
        nsteps = tf.shape(word_embedded)[1]
        output = tf.reshape(word_embedded, [-1, 2*hidden_size_word])
        pred = tf.matmul(output, W_tag) + b_tag
        self.logits = tf.reshape(pred, [-1, nsteps, len(idx2tag)])
        pred_pos = tf.matmul(output, W_pos) + b_pos
        logits_pos = tf.reshape(pred_pos, [-1, nsteps, len(idx2pos)])
        
        with tf.variable_scope("entities"):
            log_likelihood, trans_params = tf.contrib.crf.crf_log_likelihood(
            self.logits, self.labels, tf.count_nonzero(self.word_ids, 1))
        
        with tf.variable_scope("pos"):
            log_likelihood_pos, trans_params_pos = tf.contrib.crf.crf_log_likelihood(
            logits_pos, self.labels_pos, tf.count_nonzero(self.word_ids, 1))
        
        self.cost = tf.reduce_mean(-log_likelihood) + tf.reduce_mean(-log_likelihood_pos)
        self.global_step = tf.Variable(0, trainable=False)
        
        with tf.name_scope('entity-logits'):
            self.crf_decode = tf.contrib.crf.crf_decode(self.logits, 
                                                        trans_params, 
                                                        tf.count_nonzero(self.word_ids, 1))[0]
        
        with tf.name_scope('pos-logits'):
            self.crf_decode_pos = tf.contrib.crf.crf_decode(logits_pos, 
                                                        trans_params_pos, 
                                                        tf.count_nonzero(self.word_ids, 1))[0]
        
        self.optimizer = tf.train.AdamOptimizer(learning_rate).apply_gradients(clip_grads(self.cost), 
                                                                                    global_step=self.global_step)

In [10]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model(dim_word,dim_char,dropout,learning_rate,hidden_size_char,hidden_size_word,num_layers)
sess.run(tf.global_variables_initializer())

/usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/gradients_impl.py:97: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [11]:
def iter_seq(x):
    return np.array([x[i: i+seq_len] for i in range(0, len(x)-seq_len, 1)])

def to_train_seq(*args):
    return [iter_seq(x) for x in args]

def generate_char_seq(batch):
    x = [[len(idx2word[i]) for i in k] for k in batch]
    maxlen = max([j for i in x for j in i])
    temp = np.zeros((batch.shape[0],batch.shape[1],maxlen),dtype=np.int32)
    for i in range(batch.shape[0]):
        for k in range(batch.shape[1]):
            for no, c in enumerate(idx2word[batch[i,k]]):
                temp[i,k,-1-no] = char2idx[c]
    return temp

In [12]:
train_X_seq, train_Y_seq = to_train_seq(train_X, train_Y)
_, train_Y_pos = to_train_seq(train_X, train_Y_pos)
test_X_seq, test_Y_seq = to_train_seq(test_X, test_Y)
_, test_Y_pos = to_train_seq(test_X, test_Y_pos)

In [13]:
np.unique(train_Y_seq.ravel(),return_counts=True)

(array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12]),
 array([  9443, 230706,  26911,   3480,   7340,   3460,   2880,  11500,
          1700,   2100,     20,     20]))

In [14]:
np.unique(test_Y_seq.ravel(),return_counts=True)

(array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12]),
 array([  9443, 230706,  26911,   3480,   7340,   3460,   2880,  11500,
          1700,   2100,     20,     20]))

In [15]:
idx2tag

{0: 'PAD',
 1: 'LOC',
 2: 'O',
 3: 'PRN',
 4: 'ORG',
 5: 'TIME',
 6: 'ART',
 7: 'EVENT',
 8: 'NORP',
 9: 'FAC',
 10: 'LAW',
 11: 'DOC',
 12: 'I'}

In [16]:
for i in range(50):
    total_cost = 0
    for k in range(0,(train_X_seq.shape[0] // batch_size)*batch_size,batch_size):
        batch_x = train_X_seq[k:k+batch_size]
        batch_y = train_Y_seq[k:k+batch_size]
        batch_y_pos = train_Y_pos[k:k+batch_size]
        batch_x_char = generate_char_seq(batch_x)
        step, loss, _ = sess.run([model.global_step, model.cost, model.optimizer],
                                 feed_dict={model.word_ids:batch_x,
                                           model.char_ids:batch_x_char,
                                           model.labels:batch_y,
                                           model.labels_pos:batch_y_pos})
        if step % display_step == 0 or step == 1:
            print('epoch %d, step %d, loss %f'%(i+1,step,loss))
        total_cost += loss
    total_cost /= (train_X_seq.shape[0] // batch_size)
    print('epoch %d, avg loss %f'%(i+1,total_cost))

epoch 1, step 1, loss 111.521591
epoch 1, step 200, loss 47.535225
epoch 1, step 400, loss 13.889125
epoch 1, step 600, loss 17.199051
epoch 1, step 800, loss 16.708483
epoch 1, avg loss 27.311598
epoch 2, step 1000, loss 7.558507
epoch 2, step 1200, loss 12.635284
epoch 2, step 1400, loss 8.542729
epoch 2, step 1600, loss 22.633131
epoch 2, step 1800, loss 14.607322
epoch 2, avg loss 13.833051
epoch 3, step 2000, loss 21.798588
epoch 3, step 2200, loss 6.216496
epoch 3, step 2400, loss 9.984657
epoch 3, step 2600, loss 3.206939
epoch 3, step 2800, loss 12.994102
epoch 3, avg loss 10.720457
epoch 4, step 3000, loss 11.954395
epoch 4, step 3200, loss 1.666191
epoch 4, step 3400, loss 3.858815
epoch 4, step 3600, loss 9.973527
epoch 4, avg loss 8.725374
epoch 5, step 3800, loss 10.386351
epoch 5, step 4000, loss 9.538898
epoch 5, step 4200, loss 0.499785
epoch 5, step 4400, loss 12.239088
epoch 5, step 4600, loss 4.944199
epoch 5, avg loss 7.726086
epoch 6, step 4800, loss 6.310612
epoch

epoch 43, step 39600, loss 0.735441
epoch 43, step 39800, loss 2.368262
epoch 43, step 40000, loss 0.097609
epoch 43, step 40200, loss 1.602573
epoch 43, avg loss 1.748509
epoch 44, step 40400, loss 2.188420
epoch 44, step 40600, loss 0.337450
epoch 44, step 40800, loss 1.173182
epoch 44, step 41000, loss 0.530938
epoch 44, avg loss 1.720396
epoch 45, step 41200, loss 2.173032
epoch 45, step 41400, loss 4.042316
epoch 45, step 41600, loss 3.167752
epoch 45, step 41800, loss 2.632245
epoch 45, step 42000, loss 0.763768
epoch 45, avg loss 1.786217
epoch 46, step 42200, loss 0.672659
epoch 46, step 42400, loss 0.814694
epoch 46, step 42600, loss 0.900884
epoch 46, step 42800, loss 0.469251
epoch 46, step 43000, loss 2.127480
epoch 46, avg loss 1.805542
epoch 47, step 43200, loss 0.516917
epoch 47, step 43400, loss 2.158448
epoch 47, step 43600, loss 0.315233
epoch 47, step 43800, loss 0.279900
epoch 47, avg loss 1.764071
epoch 48, step 44000, loss 1.585195
epoch 48, step 44200, loss 1.515

In [17]:
saver = tf.train.Saver(tf.global_variables())
saver.save(sess, os.getcwd()+"/concat/concat-entity-pos.ckpt")

'/home/husein/space/text-dataset/entities/bm/concat/concat-entity-pos.ckpt'

In [18]:
label_Y, label_pos, predicted_Y, predicted_pos = [], [], [], []
for k in range(0,(test_X_seq.shape[0] // batch_size)*batch_size,batch_size):
    batch_x = test_X_seq[k:k+batch_size]
    batch_x_char = generate_char_seq(batch_x)
    batch_y = test_Y_seq[k:k+batch_size]
    batch_y_pos = test_Y_pos[k:k+batch_size]
    Y_pred, Y_pos = sess.run([model.crf_decode,model.crf_decode_pos],
                  feed_dict={model.word_ids:batch_x,
                            model.char_ids:batch_x_char})
    predicted_Y.append(Y_pred)
    predicted_pos.append(Y_pos)
    label_Y.append(batch_y)
    label_pos.append(batch_y_pos)

In [19]:
print(classification_report(np.vstack(label_Y).ravel(), np.vstack(predicted_Y).ravel(), target_names=tag2idx.keys()))

             precision    recall  f1-score   support

      EVENT       0.95      0.96      0.96      9443
       TIME       0.99      0.99      0.99    230666
        ORG       0.97      0.97      0.97     26911
        ART       0.97      0.98      0.98      3480
        FAC       0.94      0.89      0.92      7340
          O       0.88      0.97      0.92      3460
        LAW       0.89      0.91      0.90      2880
        LOC       0.95      0.94      0.95     11500
       NORP       0.86      0.97      0.91      1700
        PRN       0.96      0.96      0.96      2100
        DOC       0.80      0.80      0.80        20
          I       0.87      1.00      0.93        20

avg / total       0.98      0.98      0.98    299520



/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 12, does not match size of target_names, 13
  .format(len(labels), len(target_names))


In [20]:
print(classification_report(np.vstack(label_pos).ravel(), np.vstack(predicted_pos).ravel(), target_names=pos2idx.keys()))

             precision    recall  f1-score   support

    PISAHAN       0.98      0.99      0.98    169539
   KPENEGAS       0.98      0.98      0.98      2159
       NAFI       0.99      0.92      0.95     52116
         KP       0.99      1.00      0.99     23392
       NORP       0.99      1.00      1.00      7474
         GN       0.72      0.97      0.82      4080
         KN       0.95      0.99      0.97     13200
       ARAH       0.99      0.99      0.99     12860
        KPA       1.00      1.00      1.00      5756
       SUKU       0.99      0.39      0.56       220
         NO       1.00      0.99      0.99      2880
         KH       0.97      0.86      0.91       140
         KS       0.99      0.98      0.99      2064
          O       1.00      0.99      0.99      1060
        PAD       0.98      0.98      0.98      1160
 KETERANGAN       0.95      0.95      0.95       100
         KM       0.98      0.96      0.97       600
         KJ       1.00      0.99      0.99   

/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 22, does not match size of target_names, 23
  .format(len(labels), len(target_names))


In [21]:
test_string = 'Keikhlasan merupakan faktor utama yang perlu ada pada setiap pemimpin dan ahli UMNO sekiranya mahu melihat parti itu pulih kembali selain mendapat sokongan majoriti rakyat negara ini, kata Ahli Majlis Tertinggi (MT) UMNO Datuk Seri Idris Jusoh.'

In [22]:
test_X = []
for w in test_string.split():
    w = process_word(w)
    try:
        test_X.append(word2idx[w])
    except:
        test_X.append(2)

In [23]:
batch_x_char = generate_char_seq(np.array([test_X]))

In [24]:
Y_pred = sess.run(model.crf_decode,feed_dict={model.word_ids:np.array([test_X]),
                                              model.char_ids:batch_x_char})

In [25]:
for no, i in enumerate(test_string.split()):
    print(i,idx2tag[Y_pred[0,no]])

Keikhlasan LOC
merupakan O
faktor O
utama O
yang O
perlu O
ada O
pada O
setiap O
pemimpin O
dan O
ahli O
UMNO NORP
sekiranya O
mahu O
melihat O
parti O
itu O
pulih O
kembali O
selain O
mendapat O
sokongan O
majoriti O
rakyat O
negara O
ini, O
kata O
Ahli O
Majlis ART
Tertinggi ART
(MT) O
UMNO NORP
Datuk PRN
Seri PRN
Idris PRN
Jusoh. PRN


In [26]:
def get_entity(string):
    test_X = []
    for w in string.split():
        w = process_word(w)
        try:
            test_X.append(word2idx[w])
        except:
            test_X.append(2)
    array_X = np.array([test_X])
    batch_x_char = generate_char_seq(array_X)
    Y_pred,Y_pos = sess.run([model.crf_decode,model.crf_decode_pos],feed_dict={model.word_ids:array_X,
                                              model.char_ids:batch_x_char})
    for no, i in enumerate(string.split()):
        print(i,idx2tag[Y_pred[0,no]],idx2pos[Y_pos[0,no]])

In [27]:
get_entity('KUALA LUMPUR: Sempena sambutan Aidilfitri minggu depan, Perdana Menteri Tun Dr Mahathir Mohamad dan Menteri Pengangkutan Anthony Loke Siew Fook menitipkan pesanan khas kepada orang ramai yang mahu pulang ke kampung halaman masing-masing. Dalam video pendek terbitan Jabatan Keselamatan Jalan Raya (JKJR) itu, Dr Mahathir menasihati mereka supaya berhenti berehat dan tidur sebentar  sekiranya mengantuk ketika memandu.')

KUALA LOC KN
LUMPUR: LOC KN
Sempena O KJ
sambutan O KN
Aidilfitri EVENT KN
minggu O KN
depan, O KN
Perdana PRN KN
Menteri PRN KN
Tun PRN KN
Dr PRN KN
Mahathir PRN KN
Mohamad PRN KN
dan O KH
Menteri PRN KN
Pengangkutan PRN KN
Anthony PRN KN
Loke PRN KN
Siew PRN KN
Fook PRN KN
menitipkan O KN
pesanan ART KN
khas ART KN
kepada O KS
orang O KN
ramai O KN
yang O KETERANGAN
mahu O KN
pulang O KN
ke O KS
kampung O KN
halaman O KN
masing-masing. O KN
Dalam O KN
video O KN
pendek O KN
terbitan O KN
Jabatan NORP KN
Keselamatan NORP KN
Jalan NORP KN
Raya ART KN
(JKJR) ART KN
itu, O GN
Dr PRN KN
Mahathir PRN KN
menasihati O KN
mereka O GN
supaya O KH
berhenti O KJ
berehat O KJ
dan O KH
tidur O KN
sebentar O KA
sekiranya O KJ
mengantuk O KN
ketika O KN
memandu. O KJ


In [28]:
[n.name for n in tf.get_default_graph().as_graph_def().node if "Variable" in n.op or n.name.find('Placeholder') >= 0 or n.name.find('entity-logits') == 0 or n.name.find('pos-logits') == 0]

['Placeholder',
 'Placeholder_1',
 'Placeholder_2',
 'Placeholder_3',
 'Variable',
 'Variable_1',
 'bidirectional_rnn_char_0/fw/lstm_cell/kernel',
 'bidirectional_rnn_char_0/fw/lstm_cell/bias',
 'bidirectional_rnn_char_0/bw/lstm_cell/kernel',
 'bidirectional_rnn_char_0/bw/lstm_cell/bias',
 'bidirectional_rnn_char_1/fw/lstm_cell/kernel',
 'bidirectional_rnn_char_1/fw/lstm_cell/bias',
 'bidirectional_rnn_char_1/bw/lstm_cell/kernel',
 'bidirectional_rnn_char_1/bw/lstm_cell/bias',
 'bidirectional_rnn_word_0/fw/lstm_cell/kernel',
 'bidirectional_rnn_word_0/fw/lstm_cell/bias',
 'bidirectional_rnn_word_0/bw/lstm_cell/kernel',
 'bidirectional_rnn_word_0/bw/lstm_cell/bias',
 'bidirectional_rnn_word_1/fw/lstm_cell/kernel',
 'bidirectional_rnn_word_1/fw/lstm_cell/bias',
 'bidirectional_rnn_word_1/bw/lstm_cell/kernel',
 'bidirectional_rnn_word_1/bw/lstm_cell/bias',
 'w',
 'b',
 'w_pos',
 'b_pos',
 'entities/transitions',
 'pos/transitions',
 'Variable_2',
 'entity-logits/count_nonzero/NotEqual/y',

In [29]:
# only load Variables, placeholder for input, and our logits
strings=','.join([n.name for n in tf.get_default_graph().as_graph_def().node if "Variable" in n.op or n.name.find('Placeholder') >= 0 or n.name.find('entity-logits') == 0 or n.name.find('pos-logits') == 0])

In [30]:
def freeze_graph(model_dir, output_node_names):

    if not tf.gfile.Exists(model_dir):
        raise AssertionError(
            "Export directory doesn't exists. Please specify an export "
            "directory: %s" % model_dir)

    checkpoint = tf.train.get_checkpoint_state(model_dir)
    input_checkpoint = checkpoint.model_checkpoint_path
    
    absolute_model_dir = "/".join(input_checkpoint.split('/')[:-1])
    output_graph = absolute_model_dir + "/frozen_model.pb"
    clear_devices = True
    with tf.Session(graph=tf.Graph()) as sess:
        saver = tf.train.import_meta_graph(input_checkpoint + '.meta', clear_devices=clear_devices)
        saver.restore(sess, input_checkpoint)
        output_graph_def = tf.graph_util.convert_variables_to_constants(
            sess,
            tf.get_default_graph().as_graph_def(),
            output_node_names.split(",")
        ) 
        with tf.gfile.GFile(output_graph, "wb") as f:
            f.write(output_graph_def.SerializeToString())
        print("%d ops in the final graph." % len(output_graph_def.node))

In [31]:
freeze_graph("concat", strings)

INFO:tensorflow:Restoring parameters from /home/husein/space/text-dataset/entities/bm/concat/concat-entity-pos.ckpt
INFO:tensorflow:Froze 75 variables.
Converted 75 variables to const ops.
1774 ops in the final graph.


In [32]:
def load_graph(frozen_graph_filename):
    with tf.gfile.GFile(frozen_graph_filename, "rb") as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def)
    return graph

In [33]:
g=load_graph('concat/frozen_model.pb')

In [34]:
for op in g.get_operations():
    print(op.name)

import/Placeholder
import/Placeholder_1
import/Placeholder_2
import/Placeholder_3
import/Variable
import/Variable/read
import/Variable_1
import/Variable_1/read
import/embedding_lookup
import/embedding_lookup_1
import/Shape
import/strided_slice/stack
import/strided_slice/stack_1
import/strided_slice/stack_2
import/strided_slice
import/strided_slice_1/stack
import/strided_slice_1/stack_1
import/strided_slice_1/stack_2
import/strided_slice_1
import/mul
import/strided_slice_2/stack
import/strided_slice_2/stack_1
import/strided_slice_2/stack_2
import/strided_slice_2
import/Reshape/shape/2
import/Reshape/shape
import/Reshape
import/bidirectional_rnn_char_0/fw/fw/Rank
import/bidirectional_rnn_char_0/fw/fw/range/start
import/bidirectional_rnn_char_0/fw/fw/range/delta
import/bidirectional_rnn_char_0/fw/fw/range
import/bidirectional_rnn_char_0/fw/fw/concat/values_0
import/bidirectional_rnn_char_0/fw/fw/concat/axis
import/bidirectional_rnn_char_0/fw/fw/concat
import/bidirectional_rnn_char_0/fw/fw

In [35]:
x = g.get_tensor_by_name('import/Placeholder:0')
x_char = g.get_tensor_by_name('import/Placeholder_1:0')
pos_logits = g.get_tensor_by_name('import/pos-logits/cond/Merge:0')
test_sess = tf.InteractiveSession(graph=g)
results_pos = test_sess.run(pos_logits, feed_dict={x:np.array([test_X]), x_char:batch_x_char})[0]
for i in results_pos:
    print(idx2pos[i])

KN
KJ
KN
KN
KETERANGAN
KJ
KN
KS
PISAHAN
KN
KH
KN
KN
KJ
KN
KJ
KN
GN
KPENGUAT
KA
KN
KJ
KN
KN
KN
KN
GN
KN
KN
KN
KN
KN
KN
KN
KN
KN
KN
